In [ ]:
text='From then on , the members continued their separate careers . Danny Boy founded an art company . DJ Lethal became a member of nu metal band Limp Bizkit , who would cover " Jump Around " at live concerts , particularly in Limp Bizkit \'s early years during the Family Values Tour 1998 . Everlast achieved multi-platinum solo fame in 1998 with his album \'\' Whitey Ford Sings the Blues . \'\' The first single from that album was " What It \'s Like " . In 2000 , a feud between Everlast and rapper Eminem coincided with the gold-selling \'\' Eat at Whitey \'s \'\' , which included minor hits " Black Jesus " and " Black Coffee " , and featured a collaboration with Carlos Santana . After the sale of the Tommy Boy Records \' master tapes to Warner Bros . Records , Everlast signed with Island/Def Jam , and released his \'\' White Trash Beautiful \'\' LP in 2004'

In [ ]:
from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = WikiMathsDatasetLoader()

dataset = loader.get_dataset(lags=14)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h
from tqdm import tqdm

model = RecurrentGCN(node_features=14, filters=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2023/6/24 13:56
# @Author  : gao qiang
# @Site    :
# @File    : graph
# @Project : sota_end2end_parser

'''
将RST树构造为图用GAT处理
'''
import dgl
import numpy as np
import networkx as nx


def build_graph():
    # All 78 edges are stored in two numpy arrays. One for source endpoints
    # while the other for destination endpoints.
    src = np.array([1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10,
                    10, 11, 12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21,
                    25, 25, 27, 27, 27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32,
                    32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33,
                    33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
    dst = np.array([0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4,
                    5, 0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23,
                    24, 2, 23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23,
                    29, 30, 31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30,
                    31, 32])
    # Edges are directional in DGL; Make them bi-directional.
    u = np.concatenate([src, dst])
    v = np.concatenate([dst, src])
    # Construct a DGLGraph
    # dgl.DGLGraph()
    return dgl.graph((u, v))


def graph_visualization(G: dgl.DGLGraph):
    # %matplotlib inline jupyter
    # Since the actual graph is undirected, we convert it for visualization
    # purpose.
    nx_G = G.to_networkx().to_undirected()
    # Kamada-Kawaii layout usually looks pretty for arbitrary graphs
    pos = nx.kamada_kawai_layout(nx_G)
    nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])


def process_graph():
    G = build_graph()
    print('We have %d nodes.' % G.number_of_nodes())
    print('We have %d edges.' % G.number_of_edges())
    graph_visualization(G)
process_graph()

In [ ]:
import torch
x = torch.randn((5,2,1))
print(x)
y=torch.concat((x[:][0][:],x[:][1][:]),dim=1)
print(y)
y.shape
